# ☕ PressReader Coffee Article Search with SQLite (Colab)

This notebook:

- Searches for **coffee-related articles** using the [PressReader Discovery API](https://www.pressreader.com/)
- Stores new articles in a **local SQLite database** (`pressreader_coffee_results.db`)
- Renders results in readable **Markdown format**
- Prevents duplicates using a **primary key constraint on article ID**

---

## 🔐 Setup Instructions

Before running the notebook, follow these steps in **Google Colab**:

1. Click the play button on the first code cell.
2. When prompted, enter your **PressReader API key** securely.
3. Then run the remaining cells to:
   - Fetch fresh articles
   - Store them in SQLite
   - View saved results in a readable format

### 📌 Set your API key (one-time per session):

```python
from google.colab import userdata
userdata.set_secret('PRESSREADER_API_KEY')

## ⚙️ Configuration
Set the database filename, API endpoint, and search query here.

In [ ]:
# --- Configuration --
DB_FILE = "pressreader_coffee_results.db"
API_URL = 'https://api.prod.pressreader.com/discovery/v1/search?offset=0&limit=5'
QUERY = 'coffee'

## 🚀 Execution
The main script logic is below. It imports dependencies, defines functions, and runs the process.

In [ ]:
import os
import requests
import json
import sqlite3
from IPython.display import Markdown, display
from dotenv import load_dotenv

def get_api_key():
    """Fetches the PressReader API key from Colab secrets or a .env file."""
    try:
        from google.colab import userdata
        api_key = userdata.get('PRESSREADER_API_KEY')
        if not api_key:
            raise ValueError("❌ PRESSREADER_API_KEY not found in Colab secrets.")
        print("🔑 API key loaded from Colab secrets.")
        return api_key
    except ImportError:
        load_dotenv()  # Load environment variables from .env file
        api_key = os.environ.get('PRESSREADER_API_KEY')
        if not api_key:
            raise ValueError("❌ PRESSREADER_API_KEY not found. Create a .env file or set it as an environment variable.")
        print("🔑 API key loaded from .env file.")
        return api_key

def fetch_articles(api_key, query, api_url):
    """Fetches articles from the PressReader API."""
    headers = {'Content-Type': 'application/json', 'api-key': api_key}
    payload = {'query': query}
    response = requests.post(api_url, headers=headers, json=payload)
    response.raise_for_status()  # Raises an exception for bad status codes
    return response.json().get("items", [])

def init_db(conn):
    """Initializes the SQLite database and creates the articles table."""
    with conn:
        conn.execute("""
        CREATE TABLE IF NOT EXISTS articles (
            id TEXT PRIMARY KEY,
            title TEXT,
            description TEXT,
            publication TEXT,
            date TEXT,
            url TEXT
        )
        """)

def save_articles(conn, articles):
    """Saves a list of articles to the database, skipping duplicates."""
    inserted_count = 0
    with conn:
        for article in articles:
            try:
                article_id = str(article.get("article", {}).get("id", "no-id"))
                title = article.get("article", {}).get("title", "Untitled")
                url = article.get("article", {}).get("url", "")
                description = article.get("summary", "")
                publication = article.get("publication", {}).get("title", "Unknown")
                date = article.get("issue", {}).get("date", "").split("T")[0]
                
                conn.execute("""
                INSERT INTO articles (id, title, description, publication, date, url)
                VALUES (?, ?, ?, ?, ?, ?)
                """, (article_id, title, description, publication, date, url))
                inserted_count += 1
            except sqlite3.IntegrityError:
                pass # Article already exists
    return inserted_count

def format_articles_as_markdown(articles):
    """Formats a list of articles into a Markdown string."""
    markdown_lines = ["# ☕ Coffee Articles from PressReader\n"]
    for i, article in enumerate(articles, start=1):
        title = article.get("article", {}).get("title", "Untitled")
        url = article.get("article", {}).get("url", "")
        publication = article.get("publication", {}).get("title", "Unknown")
        date = article.get("issue", {}).get("date", "").split("T")[0]
        description = article.get("summary", "")
        
        markdown_lines.append(f"### {i}. [{title}]({url})")
        markdown_lines.append(f"*Published in: **{publication}** on {date}*  ")
        if description:
            markdown_lines.append(f"> {description}\n")
    return "\n".join(markdown_lines)

# --- Main Execution ---
try:
    api_key = get_api_key()
    articles = fetch_articles(api_key, QUERY, API_URL)
    
    with sqlite3.connect(DB_FILE, timeout=10) as conn:
        init_db(conn)
        inserted_count = save_articles(conn, articles)
    
    markdown_output = format_articles_as_markdown(articles)
    display(Markdown(markdown_output))
    print(f"✅ Inserted {inserted_count} new article(s) into {DB_FILE}")

except (ValueError, requests.exceptions.RequestException) as e:
    print(f"❌ An error occurred: {e}")


# ☕ Coffee Articles from PressReader

### 1. [What is brew­ing Singa­pore's cof­fee boom?](https://www.pressreader.com/singapore/the-edge-singapore/20250728/281792815078682)
*Published in: **The Edge Singapore** on 2025-07-28*  
> The Economist Intelligence Unit (EIU) says that Singapore is the world’s most expensive city.
A cup of Kopi-O (coffee without milk) costs about $1, which is about US$0.70.
Malaysia’s Kopi-O costs $0.50 and the GDP per capita is US$12,000.

### 2. [Star­bucks, Dunkin’ lead Phl cof­fee mar­ket; local brands catch­ing up](https://www.pressreader.com/philippines/the-philippine-star/20250728/281925959063733)
*Published in: **The Philippine Star** on 2025-07-28*  
> brands continue to dominate the country’s coffee industry, but homegrown brands are gaining traction, helping to diversify and enrich Filipinos’ caffeine options, according to the University of Asia and the Pacific-Center for Food and Agri Business (UA&P-CFA).
In a policy brief, UA&P-CFA said the local café market is led by a “few dominant” global brands, based on a
The policy brief indicated that Starbucks Corp. remains as the industry leader, accounting for almost 54 percent of total value share among chain café and bar operators.

### 3. [TCPL pre­pares for another quarter of low cof­fee mar­gins](https://www.pressreader.com/india/mint-mumbai/20250724/281878714416579)
*Published in: **Mint Mumbai** on 2025-07-24*  
> Packaged food and beverage firm Tata Consumer Products Ltd (TCPL) on Wednesday said coffee prices are currently on a downward trajectory, though the ongoing volatility necessitates close monitoring following a period of significant surge.
In the March quarter, TCPL had said prices of Arabica coffee were 97% higher year-onyear, while Robusta coffee prices were up 56%.
Despite a global price drop impacting profitability, the firm’s coffee business in India reported revenue growth of 67% on the back of a 33rise % in volumes.

### 4. [TCPL pre­pares for another quarter of low cof­fee mar­gins](https://www.pressreader.com/india/mint-hyderabad/20250724/281814289907148)
*Published in: **Mint Hyderabad** on 2025-07-24*  
> Packaged food and beverage firm Tata Consumer Products Ltd (TCPL) on Wednesday said coffee prices are currently on a downward trajectory, though the ongoing volatility necessitates close monitoring following a period of significant surge.
In the March quarter, TCPL had said prices of Arabica coffee were 97% higher year-onyear, while Robusta coffee prices were up 56%.
Despite a global price drop impacting profitability, the firm’s coffee business in India reported revenue growth of 67% on the back of a 33rise % in volumes.

### 5. [TCPL pre­pares for another quarter of low cof­fee mar­gins](https://www.pressreader.com/india/mint-delhi/20250724/281844354678219)
*Published in: **Mint Delhi** on 2025-07-24*  
> Packaged food and beverage firm Tata Consumer Products Ltd (TCPL) on Wednesday said coffee prices are currently on a downward trajectory, though the ongoing volatility necessitates close monitoring following a period of significant surge.
In the March quarter, TCPL had said prices of Arabica coffee were 97% higher year-onyear, while Robusta coffee prices were up 56%.
Despite a global price drop impacting profitability, the firm’s coffee business in India reported revenue growth of 67% on the back of a 33rise % in volumes.


✅ Inserted 5 new article(s) into SQLite (pressreader_coffee_results.db)


## 🔍 View Saved PressReader Coffee Articles

This code connects to the local SQLite database (`pressreader_coffee_results.db`) and retrieves all stored articles related to **coffee**. It displays them in reverse chronological order (most recent first), showing:

- Article ID
- Title
- Summary/Description
- Publication name
- Publication date
- Direct URL to the article

Use this to **review all previously stored articles** in a readable format.


In [ ]:
import sqlite3  # Import the built-in library to interact with SQLite databases

# ---------------------------------------------
# STEP 1: Connect to the SQLite database file
# ---------------------------------------------
# If the file doesn't exist, it will be created automatically.
# In this case, it should already exist from previous runs where articles were inserted.
conn = sqlite3.connect("pressreader_coffee_results.db")
cursor = conn.cursor()

# ---------------------------------------------
# STEP 2: Query the articles table for data
# ---------------------------------------------
# This SQL query selects 6 fields from the `articles` table:
# - id: the unique article ID (used as the primary key)
# - title: the article's title
# - description: a brief summary or excerpt
# - publication: the name of the publication that published it
# - date: the publication date (in YYYY-MM-DD format)
# - url: direct link to the article online
#
# The results are filtered to exclude empty or null dates,
# and sorted by `date` in descending order, so the most recent articles appear first.
cursor.execute("""
    SELECT id, title, description, publication, date, url
    FROM articles
    WHERE date IS NOT NULL AND date != ''
    ORDER BY date DESC
""")

# Fetch all the resulting rows and store them in a list called `rows`
rows = cursor.fetchall()

# ---------------------------------------------
# STEP 3: Loop through the results and display them
# ---------------------------------------------
# For each row (article), print it out in a clean, readable format.
# The fields are displayed with clear labels, and each article is numbered.
for idx, row in enumerate(rows, 1):  # Start numbering from 1
    print(f"\n--- Article {idx} ---")
    print(f"ID:          {row[0]}")
    print(f"Title:       {row[1]}")
    print(f"Description: {row[2]}")
    print(f"Publication: {row[3]}")
    print(f"Date:        {row[4]}")
    print(f"URL:         {row[5]}")

# ---------------------------------------------
# STEP 4: Close the database connection
# ---------------------------------------------
# This is good practice to release system resources and ensure changes are saved.
conn.close()


--- Article 1 ---
ID:          281792815078682
Title:       What is brew­ing Singa­pore's cof­fee boom?
Description: The Economist Intelligence Unit (EIU) says that Singapore is the world’s most expensive city.
A cup of Kopi-O (coffee without milk) costs about $1, which is about US$0.70.
Malaysia’s Kopi-O costs $0.50 and the GDP per capita is US$12,000.
Publication: The Edge Singapore
Date:        2025-07-28
URL:         https://www.pressreader.com/singapore/the-edge-singapore/20250728/281792815078682

--- Article 2 ---
ID:          281925959063733
Title:       Star­bucks, Dunkin’ lead Phl cof­fee mar­ket; local brands catch­ing up
Description: brands continue to dominate the country’s coffee industry, but homegrown brands are gaining traction, helping to diversify and enrich Filipinos’ caffeine options, according to the University of Asia and the Pacific-Center for Food and Agri Business (UA&P-CFA).
In a policy brief, UA&P-CFA said the local café market is led by a “few dominant” glo